# Grand Challenge API Client: interact with grand-challenge.org via python
## Chapter: Algorithms


<img src="pipeline.png" />

GC-API provides a handy functionality to interract with grand-challenge.org via python.

Its purpose and broader functionality is discussed in details in [this](https://grand-challenge.org/blogs/grand-challenge-api-client/) blogpost. In order to use gc-api you will also need to obtain a personal API token. The blog above describe what it is and where to find it. 


In this tutorial we will focus on grand-challenge Algorithms and in details go over following steps:

1. [Uploading input to Algorithms on grand-challenge.org for inference.](#section_1)
2. [Downloading inference results from an Algorithm on grand-challenge.org.](#section_2)
3. [Uploading multiple item input to an Algorithm on grand-challenge.org.](#section_3)
4. [Downloading the results of an algorithm that produces multiple item output.](#section_4)

Remember that you need to request a permission prior to using an algorithm. You do not need to request a permission if you are using your own algorithm.

In [ ]:
pip install gcapi --upgrade

In [ ]:
#import nessesary libraries
import gcapi
from pathlib import Path
from tqdm import tqdm
import SimpleITK as sitk
import numpy as np
import os


We use the API token to authentificate to grand-challenge platform via API

In [ ]:
#authorise with your personal token
my_personal_GC_API_token = ''
client = gcapi.Client(token=my_personal_GC_API_token)

## [1. Uploading input to Algorithms on grand-challenge.org for inference.](#section_1)

In this section, we will use [Pulmonary Lobe Segmentation](https://grand-challenge.org/algorithms/pulmonary-lobe-segmentation/) by Weiyi Xie. This algorithm performs automatic segmentation of pulmonary lobes of a given chest CT scan. The algorithm uses a contextual two-stage U-Net architecture. We will use example chest CT scans from [coronacases.org](coronacases.org). They are anonimized.
We will now upload the CT scans of Covid-19 patients to Pulmonary Lobe Segmentation algorithm on grand-challenge using GC-API.

In [ ]:
# initialize the algorithm, providing a slug
algorithm_1 = client.algorithms.detail(slug="pulmonary-lobe-segmentation")

In [ ]:
# explore, which input the algorithm expects
algorithm_1["inputs"]

#### Submit the inputs to the algorithm one by one

Grand-challenge creats a job instance for each set of inputs. To create a job instance use a command:

    job = client.run_external_job(algorithm="slug-of-the-algorithm", inputs={ "interface": [ file ] }),
    
where argument "algorithm" expects a str with a slug of the algorithm you want to use and argument "inputs" expects a dictionary where keys are expected interfaces and the file is str path/url to a particular input file. 

Be aware that with this version (0.5.0) the input file path/url needs be placed into a list.

In [ ]:
# get the path to the files
files = ["io/case01.mha", "io/case02.mha"]
#timeout
jobs = []

# submit a job for each file in your file list
for file in files:
    
    job = client.run_external_job(
        algorithm="pulmonary-lobe-segmentation",
        inputs={
            "generic-medical-image": [Path(file)]
        }
    )
    jobs.append(job)

#### Get the statuses of the submitted jobs

In [ ]:
jobs = [client.algorithm_jobs.detail(job["pk"]) for job in jobs]

print([job["status"] for job in jobs])

After all of your jobs have ended up with a status 'Succeeded', you can download the results. You can also use infer the Algorithm on existing Archive on grand-challenge.org (if you have been granted access to it).

## [2. Downloading inference results from an Algorithm on grand-challenge.org.](#section_2)

#### Download the results from the algorithm

In [ ]:
# loop through input files 
for job, input_fname in tqdm(zip(jobs, files)):
    
    # loop through job outputs 
    for output in job["outputs"]:
        
        # check whether if output exists
        if output["image"] is not None:
            
            # get image details
            image_details = client(url=output["image"])
            print('image_details',image_details)
            for file in image_details["files"]:
                print('file',file)
                # create the output filename
                output_file = Path(input_fname.replace(".mha", "_lobes.mha"))
                if output_file.suffix != ".mha":
                    raise ValueError("Output file needs to have .mha extension")
                output_file.parent.mkdir(parents=True, exist_ok=True)
                with output_file.open("wb") as fp:
                    # get the impage from url and write it 
                    response = client(url = file["file"],follow_redirects=True).content
                    fp.write(response)

## [3. Uploading multiple item input to an Algorithm on grand-challenge.org.](#section_3)


In this section we will take a look, on how to upload multiple item input to an Algorithm on grand-challenge.org. Asw an example we will use Alessa Herrings Algorithm - [Deep Learning-Based Lung Registration](https://grand-challenge.org/algorithms/deep-learning-based-ct-lung-registration/).

This algorithm requires the following inputs:
1. fixed image (CT)
2. fixed mask (lungs segmentation)
3. moving image (CT)
4. moving mask (lungs segmentation)

We will use the scans from the previous section as well as the Algorithm output (lung lobes segmentation) in this section. Therefore, we will have to binarize the lobe masks and create lung masks

#### Binarize the masks obtained from previous example

In [ ]:
# provide paths of the lobe segmentations
lobes = [
    "io/case01_lobes.mha",
    "io/case02_lobes.mha",
]
#loop through the files
for lobe_file in lobes:
    #read image with sitk
    lobe = sitk.ReadImage(lobe_file)
    origin, spacing, direction = lobe.GetOrigin(), lobe.GetSpacing(), lobe.GetDirection()
    lobe = sitk.GetArrayFromImage(lobe)
    # binarize
    lobe[lobe > 1] = 1
    lungs = lobe.astype(np.uint8)
    lungs = sitk.GetImageFromArray(lungs)
    lungs.SetOrigin(origin)
    lungs.SetSpacing(spacing)
    lungs.SetDirection(direction)
    #write the modified image back into file
    sitk.WriteImage(lungs, lobe_file.replace("_lobes", "_lungs"), True)

In [ ]:
# initialize the algorithm
algorithm_2 = client.algorithms.detail(slug="deep-learning-based-ct-lung-registration")

#### Inspecting the algorithm object
You can inspect the algorithm object to understand what kind of inputs it requires

In [ ]:
algorithm_2["inputs"]

#### Submit the inputs to the algorithm

In [ ]:
# create a job
registration_job = client.run_external_job(
    algorithm="deep-learning-based-ct-lung-registration",
    inputs={
        "fixed-image": [Path("io/case01.mha")],
        "moving-image": [Path("io/case02.mha")],
        "fixed-mask": [Path("io/case01_lungs.mha")],
        "moving-mask": [Path("io/case02_lungs.mha")],
    }
)

#### Get the status of the job

In [ ]:
registration_job = client.algorithm_jobs.detail(registration_job["pk"])
registration_job["status"]

#### Download the results

After the status of the job is `Succeeded`, you can procceed to downloading the result.

In [ ]:
# loop through the outputs 
for output in registration_job["outputs"]:
    print('output',output)
    # get image details
    image_details = client(url=output["image"])
    output_slug = output["interface"]["slug"]
    print("Downloading", output_slug)
    for file in image_details["files"]:
        output_file = Path(f"{output_slug}.mha")
        output_file.parent.mkdir(parents=True, exist_ok=True)
        with output_file.open("wb") as fp:
          
            fp.write(client(url=file["file"], follow_redirects=True).content)

Note that both these algorithms wrote `.mha` files as outputs. For algorithms that require different outputs, you can loop through the outputs of a successful job and search under "interface", which will tell you what kind of outputs you will have to download

### [4. Downloading the results of an algorithm that produces multiple item output.](#section_4)

In this section we will focus on how to download  results from an algorithm that produces multiple outputs. We will use the algorithm for pulmonary lobes segmentation Covid-19 cases. This algorithm output the segmentation for a particular input as well as a "screenshot" of a middle slice for rapid inspection of algorithm performance.


In [ ]:
# initialize the algorithm, providing a slug
algorithm_4 = client.algorithms.detail(slug="pulmonary-lobe-segmentation-for-covid-19-ct-scans")

# explore, which input the algorithm expects
algorithm_4["inputs"]

We will, firstly infer the algorithm on the existing images, exactly the same way we did before.

In [ ]:
from gcapi import Client
c = Client(token=my_personal_GC_API_token)

#name of the archive 
archive_slug = "coronacases.org"
#save path on your machine
output_archive_dir = 'output_scans'
outputarchivedir_screenshots = 'output_screenshots'

archives = c(url="https://grand-challenge.org/api/v1/archives/")["results"]

corona_archive = None
for archive in archives:
    if archive["name"] == archive_slug:
        corona_archive = archive
        break
if corona_archive is None:
    raise Exception("archive not found on GC")

In [ ]:
print(corona_archive)

# get information about images in archive from GC API
params = {
    'archive': corona_archive['id'],
}
response = c(url="https://grand-challenge.org/api/v1/cases/images/", params=params)
#print(response)
urls=[]
for r in response['results']:
    urls.append(r['api_url'])
 

In [ ]:
print(len(urls))

In [ ]:
print(urls)

In [ ]:
# initialize the archive, providing a slug
jobs = []

# submit a job for each file in your file list
for url in urls[:2]:
    print(url)
    job = client.run_external_job(
        algorithm="pulmonary-lobe-segmentation-for-covid-19-ct-scans",
        inputs={
            "ct-image": url
        }
    )
    jobs.append(job)

Lets check the status of the job.

In [ ]:
jobs = [client.algorithm_jobs.detail(job["pk"]) for job in jobs]

print([job["status"] for job in jobs])

If the job status is 'Succeeded' we can procceed to downloading the results. In this part we will go through scenario where we no longer have the details of the particular job that processed algorithm and inputs.

In [ ]:
from gcapi import Client
c = Client(token=my_personal_GC_API_token)


# get algorithm providing the slug
algorithm = "pulmonary-lobe-segmentation-for-covid-19-ct-scans"
algorithm_details = c(path="algorithms/", params={"slug": algorithm})

# extract details   
algorithm_details = algorithm_details["results"][0]
algorithm_uuid = algorithm_details["pk"]

# Define dictionaries for image uuid mappings
images_mapping = {}
images_mapping_scans = {}

# get the desired archive
archives = c(url="https://grand-challenge.org/api/v1/archives/")["results"]
archive_slug = 'coronacases.org'
target_archive = None

# loop through archives and select the one with a slug that you are looking for ('coronacases.org')
for archive in archives:
    if archive["name"] == archive_slug:
        target_archive = archive
        break



We have generated a set of outputs to a set of inputs. Now, we need to find out which output corresponds to which input. This can be figured out via unique identifiers of images. Each image in an archive has a unique identifier (uuid). Here we create a **mapping between input image names and uuids**. We collect the uuids to a list.

In [ ]:
# get uuids in archive
done = False
iteration = 0
image_uuids = []
# create a mapping between image uuids and input names
while not done:
    iteration += 1
    if iteration == 1:
        # get information about images in archive from GC API
        params = {'archive': target_archive['id']}
        response = c(url="https://grand-challenge.org/api/v1/cases/images/", params=params)
    else:
        # get information about images on next page
        response = c(url=response["next"])
    images = response['results']
    for image in images:
        # create a mapping for image uuids
        uuid = image['pk']
        images_mapping[uuid] = image['name'] + "_" + uuid
        images_mapping_scans[uuid] = image['name'] 
        image_uuids += [uuid]
    if response["next"] is None:
        # stop if no next page left
        done = True
print('image_uuids:',image_uuids)
print('--------------------------------------------------------------------------------------------------------')
print('images_mapping_scans:',images_mapping_scans)

Now, we will loop through uuids and collect the algorithm job details corresponding to each unique image identifier.

In [ ]:
# get algorithm results for the image
output_image_files = []
screenshot_files = []
counter = 0
# loop through uuid
for image_uuid in image_uuids:
    params = {'algorithm_image__algorithm': algorithm_uuid, 'input_image': image_uuid}
    # get the jobs details corresponding to a particular uuid and algorithm
    results = c.algorithm_jobs.iterate_all(params)
    #iterate through the results
    for result in results:
        counter += 1
        print('--------------------------------------------------------------------------------------------------------')
        #iterate through the outputs
        for output in result['outputs']:
            # here we go over different interfaces and write the corresponding output
            print('interface:',output["interface"]["slug"])
            if output["interface"]["slug"] == "pulmonary-lobes":
                image = c(url=output['image'])
                for file in image["files"]:
                    if file['image_type'] == "MHD":
                        new_file = file['file']#image['files'][0]['file']
                        output_image_files += [new_file]
                        dest_path_mha = Path(os.path.join(output_archive_dir, images_mapping_scans[image_uuid]))
                        with open(dest_path_mha, 'wb') as f1:
                            response_1 = c(url=new_file, follow_redirects=True)
                            f1.write(response_1.content)
                            print(dest_path_mha)
            if output["interface"]["slug"] == "pulmonary-lobes-screenshot":
                image = c(url=output['image'])
                for file in image["files"]:
                    if file['image_type'] == "TIFF":
                        new_file = file['file']
                        screenshot_files += [new_file]
                        dest_path = os.path.join(outputarchivedir_screenshots, images_mapping[image_uuid] + '.tif')
                        with open(dest_path, 'wb') as f1:
                            response_2 = c(url=new_file, follow_redirects=True)
                            f1.write(response_2.content)
                            print(dest_path)